In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd ./drive/MyDrive/최애림/cakd7/m9_딥러닝/3차 프로젝트/5팀(윤정준, 최애림, 진청아, 이정현a)/코드
# %cd ./drive/MyDrive/5팀(윤정준, 최애림, 진청아, 이정현a)/코드
# %cd ./drive/MyDrive/5팀(윤정준, 최애림, 진청아, 이정현a)/코드

/content/drive/.shortcut-targets-by-id/1niQkF2oFYkuV2aahMfHbpa3kXs5yrcAd/5팀(윤정준, 최애림, 진청아, 이정현a)/코드


## 필요 라이브러리 임포트

In [ ]:
import os
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

#원본 이미지
train_images = glob("./train_imgs/*.png")
len(train_images)

1000

In [ ]:
# Install the OpenSlide C library and Python bindings
!apt-get install openslide-tools
!apt-get install python3-openslide

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openslide-tools is already the newest version (3.4.1+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-openslide is already the newest version (1.1.1-2ubuntu4).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import random
import shutil
import time
import numpy as np
from openslide import open_slide, __library_version__ as openslide_version
from PIL import Image
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split

!pip install ipython-autotime

%load_ext autotime
%tensorflow_version 2.x
import tensorflow as tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
time: 3.78 s (started: 2022-11-25 07:17:54 +00:00)


## 전체 파일 경로 지정

In [ ]:
# 전체 학습 이미지 경로 지정
## 베이스 경로
base_dir = './train_imgs/'

## 파일이름 리스트
slide_files = os.listdir(base_dir)

## 전체경로 리스트
slide_paths = [base_dir + slide_file for slide_file in slide_files]
slide_paths = sorted(slide_paths)
slide_paths[:10]

['./train_imgs/BC_01_0001.png',
 './train_imgs/BC_01_0002.png',
 './train_imgs/BC_01_0003.png',
 './train_imgs/BC_01_0004.png',
 './train_imgs/BC_01_0005.png',
 './train_imgs/BC_01_0006.png',
 './train_imgs/BC_01_0007.png',
 './train_imgs/BC_01_0008.png',
 './train_imgs/BC_01_0009.png',
 './train_imgs/BC_01_0010.png']

time: 12 ms (started: 2022-11-25 07:17:58 +00:00)


In [ ]:
slide_paths[0]

'./train_imgs/BC_01_0001.png'

time: 3.26 ms (started: 2022-11-25 07:17:58 +00:00)


# TEST

## 패치화 사용자함수

### 진행하기

In [ ]:
# openslide 이미지파일을 배열로 변환해주는 사용자함수
def read_slide(slide, x, y, level, width, height, as_float=False):
    # openslide 형식의 이미지 파일의 x,y 좌표값에서 해당 너비, 높이만큼 읽기
    im = slide.read_region((x,y), level, (width, height))

    # 'RGB'형식으로 변환
    im = im.convert('RGB') # drop the alpha channel
    
    if as_float:
        im = np.asarray(im, dtype = np.float32)
    else:
        im = np.asarray(im)
    
    # 지정된 높이, 너비의 RGB 배열인지 확인
    assert im.shape == (height, width, 3)
    
    return im

# WSI 이미지 불러와서 패치로 쪼개기 위한 조건을 정해주고 패치화하는 사용자함수
def make_patch_set_all(slide_paths, patch_size = 32):
    
    # 슬라이드 패치의 라벨값과 사이즈를 저장해줄 빈 리스트 두개 생성 
    slide_patches_all = []
    slide_patches_size = []
    

    #openslide 이미지 차례대로 for문 돌려서 패치화 해주기
    for slide_path in slide_paths[:2]:
        print(f'******{slide_path} 진행중*******')

        # 하나의 원본 사진을 openslide형태로 바꿔줌
        slide_whole = open_slide(slide_path)

        # 이미지 크기에 맞춰서 정해진 패치 사이즈 (32*32)에 해당하는 step 지정
        n_step_width = slide_whole.level_dimensions[0][0] // patch_size
        n_step_height = slide_whole.level_dimensions[0][1] // patch_size
        

        slide_patches_size.append((n_step_width, n_step_height))

        #이건 아직 잘 모르겠음 
        downsample_factor = slide_whole.level_downsamples[0]  

        
        slide_patches = []                
        for step_height in range(n_step_height):
            for step_width in range(n_step_width):

                #우리 CNN에서 maxpooling할 때 처럼 이미지를 아래 창문 크기 사이즈로 옆으로 움직이는 모습을 상상하면 될듯 
                slide_window = [int(patch_size * step_width * downsample_factor), 
                                int(patch_size * step_height * downsample_factor)] 

                #위에 생성해준 read_slide 사용자 함수를 이용하여 패치당 들어가는 픽셀 값들을 배열로 나타내줌
                # Slide 패치들 생성 
                
                slide_patch = read_slide(slide_whole, 
                                         x = slide_window[0], 
                                         y = slide_window[1], 
                                         level = 0, 
                                         width = patch_size, 
                                         height = patch_size)  
                print(f'******{step_width}_{step_height} 진행중*******') 

                slide_patches.append(slide_patch)
        slide_patches_all.append(slide_patches)

    #순서대로 나열되기 때문에, slide_patches[0]의 라벨값은 tumor_indicator에서 같은 인덱스, tumor_indicator[0] 이게 된다.
    print(slide_patches_all, slide_patches_size)
    return slide_patches_all, slide_patches_size

time: 3.11 ms (started: 2022-11-22 10:06:59 +00:00)


In [ ]:
slide_patches_all, slide_patches_size = make_patch_set_all(slide_paths, patch_size = 32)

In [ ]:
slide_patches_all[0], slide_patches_size[0]

([array([[[244, 244, 244],
          [244, 244, 244],
          [244, 244, 244],
          ...,
          [244, 244, 244],
          [244, 244, 244],
          [244, 244, 244]],
  
         [[244, 244, 244],
          [244, 244, 244],
          [244, 244, 244],
          ...,
          [244, 244, 244],
          [244, 244, 244],
          [244, 244, 244]],
  
         [[244, 244, 244],
          [244, 244, 244],
          [244, 244, 244],
          ...,
          [244, 244, 244],
          [244, 244, 244],
          [244, 244, 244]],
  
         ...,
  
         [[244, 244, 244],
          [244, 244, 244],
          [244, 244, 244],
          ...,
          [244, 244, 244],
          [244, 244, 244],
          [244, 244, 244]],
  
         [[244, 244, 244],
          [244, 244, 244],
          [244, 244, 244],
          ...,
          [244, 244, 244],
          [244, 244, 244],
          [244, 244, 244]],
  
         [[244, 244, 244],
          [244, 244, 244],
          [244, 244, 244

time: 556 ms (started: 2022-11-22 07:49:56 +00:00)


## 저장하기

In [ ]:
np.save('./array/1121_real_invasive_indicator_train_2개.npy', slide_patches_all) 
np.save('./array/1121_real_invasive_indicator_train_size_2개.npy', slide_patches_size)

time: 143 ms (started: 2022-11-22 05:59:01 +00:00)


/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


# 전체

## 패치화 사용자함수

In [ ]:
# openslide 이미지파일을 배열로 변환해주는 사용자함수
def read_slide(slide, x, y, level, width, height, as_float=False):
    # openslide 형식의 이미지 파일의 x,y 좌표값에서 해당 너비, 높이만큼 읽기
    im = slide.read_region((x,y), level, (width, height))

    # 'RGB'형식으로 변환
    im = im.convert('RGB') # drop the alpha channel
    
    if as_float:
        im = np.asarray(im, dtype = np.float32)
    else:
        im = np.asarray(im)
    
    # 지정된 높이, 너비의 RGB 배열인지 확인
    assert im.shape == (height, width, 3)
    
    return im

# WSI 이미지 불러와서 패치로 쪼개기 위한 조건을 정해주고 패치화하는 사용자함수
def make_patch_set_all(slide_paths, patch_size = 32):
    
    # 슬라이드 패치의 라벨값과 사이즈를 저장해줄 빈 리스트 두개 생성 
    slide_patches_all = []
    slide_patches_size = []
    

    #openslide 이미지 차례대로 for문 돌려서 패치화 해주기
    for slide_path in slide_paths[:50]:
        print(f'******{slide_path} 진행중*******')

        # 하나의 원본 사진을 openslide형태로 바꿔줌
        slide_whole = open_slide(slide_path)

        # 이미지 크기에 맞춰서 정해진 패치 사이즈 (32*32)에 해당하는 step 지정
        n_step_width = slide_whole.level_dimensions[0][0] // patch_size
        n_step_height = slide_whole.level_dimensions[0][1] // patch_size
        

        slide_patches_size.append((n_step_width, n_step_height))

        #이건 아직 잘 모르겠음 
        downsample_factor = slide_whole.level_downsamples[0]  

        
        slide_patches = []                
        for step_width in range(n_step_width):
            for step_height in range(n_step_height):

                #우리 CNN에서 maxpooling할 때 처럼 이미지를 아래 창문 크기 사이즈로 옆으로 움직이는 모습을 상상하면 될듯 
                slide_window = [int(patch_size * step_width * downsample_factor), 
                                int(patch_size * step_height * downsample_factor)] 

                #위에 생성해준 read_slide 사용자 함수를 이용하여 패치당 들어가는 픽셀 값들을 배열로 나타내줌
                # Slide 패치들 생성 
                
                slide_patch = read_slide(slide_whole, 
                                         x = slide_window[0], 
                                         y = slide_window[1], 
                                         level = 0, 
                                         width = patch_size, 
                                         height = patch_size)  
                # print(f'******{step_width}_{step_height} 진행중*******') 

                slide_patches.append(slide_patch)
        slide_patches_all.append(slide_patches)

    #순서대로 나열되기 때문에, slide_patches[0]의 라벨값은 tumor_indicator에서 같은 인덱스, tumor_indicator[0] 이게 된다.
    print(slide_patches_all, slide_patches_size)
    return slide_patches_all, slide_patches_size

time: 5.83 ms (started: 2022-11-23 03:35:46 +00:00)


In [ ]:
slide_patches_all, slide_patches_size = make_patch_set_all(slide_paths, patch_size = 32)

******./train_imgs/BC_01_0001.png 진행중*******
******./train_imgs/BC_01_0002.png 진행중*******
******./train_imgs/BC_01_0003.png 진행중*******
******./train_imgs/BC_01_0004.png 진행중*******
******./train_imgs/BC_01_0005.png 진행중*******
******./train_imgs/BC_01_0007.png 진행중*******
******./train_imgs/BC_01_0006.png 진행중*******
******./train_imgs/BC_01_0008.png 진행중*******
******./train_imgs/BC_01_0010.png 진행중*******
******./train_imgs/BC_01_0009.png 진행중*******
******./train_imgs/BC_01_0013.png 진행중*******


In [ ]:
slide_patches_all[0], slide_patches_size[0]

## 저장하기

In [ ]:
np.save('./array/1123_real_train_1_50.npy', slide_patches_all) 
np.save('./array/1123_real_train_size_1_50.npy', slide_patches_size)

# 전체를 쪼개서 저장하기

In [ ]:
# openslide 이미지파일을 배열로 변환해주는 사용자함수
def read_slide(slide, x, y, level, width, height, as_float=False):
    # openslide 형식의 이미지 파일의 x,y 좌표값에서 해당 너비, 높이만큼 읽기
    im = slide.read_region((x,y), level, (width, height))

    # 'RGB'형식으로 변환
    im = im.convert('RGB') # drop the alpha channel
    
    if as_float:
        im = np.asarray(im, dtype = np.float32)
    else:
        im = np.asarray(im)
    
    # 지정된 높이, 너비의 RGB 배열인지 확인
    assert im.shape == (height, width, 3)
    
    return im


# WSI 이미지 불러와서 패치로 쪼개기 위한 조건을 정해주고 패치화하는 사용자함수
def make_patch_set_all(slide_paths, patch_size = 32):
    
    # 슬라이드 패치의 라벨값과 사이즈를 저장해줄 빈 리스트 두개 생성 
    slide_patches_all = []
    slide_patches_size = []
    

    #openslide 이미지 차례대로 for문 돌려서 패치화 해주기
    for slide_path in slide_paths[:50]:
        print(f'******{slide_path} 진행중*******')

        # 하나의 원본 사진을 openslide형태로 바꿔줌
        slide_whole = open_slide(slide_path)

        # 이미지 크기에 맞춰서 정해진 패치 사이즈 (32*32)에 해당하는 step 지정
        n_step_width = slide_whole.level_dimensions[0][0] // patch_size
        n_step_height = slide_whole.level_dimensions[0][1] // patch_size
        

        slide_patches_size.append((n_step_width, n_step_height))

        #이건 아직 잘 모르겠음 
        downsample_factor = slide_whole.level_downsamples[0]  

        
        slide_patches = []                
        for step_width in range(n_step_width):
            for step_height in range(n_step_height):

                #우리 CNN에서 maxpooling할 때 처럼 이미지를 아래 창문 크기 사이즈로 옆으로 움직이는 모습을 상상하면 될듯 
                slide_window = [int(patch_size * step_width * downsample_factor), 
                                int(patch_size * step_height * downsample_factor)] 

                #위에 생성해준 read_slide 사용자 함수를 이용하여 패치당 들어가는 픽셀 값들을 배열로 나타내줌
                # Slide 패치들 생성 
                
                slide_patch = read_slide(slide_whole, 
                                         x = slide_window[0], 
                                         y = slide_window[1], 
                                         level = 0, 
                                         width = patch_size, 
                                         height = patch_size)  
#                 print(f'******{step_width}_{step_height} 진행중*******') 

                slide_patches.append(slide_patch.tolist())

        slide_patches_all.append(slide_patches)
        
    #순서대로 나열되기 때문에, slide_patches[0]의 라벨값은 tumor_indicator에서 같은 인덱스, tumor_indicator[0] 이게 된다.
    print(slide_patches_all, slide_patches_size)
    return np.array(slide_patches_all), slide_patches_size

time: 3.09 ms (started: 2022-11-23 04:56:30 +00:00)


In [ ]:
slide_patches_all, slide_patches_size = make_patch_set_all(slide_paths, patch_size = 32)

******./train_imgs/BC_01_0001.png 진행중*******
******./train_imgs/BC_01_0002.png 진행중*******
******./train_imgs/BC_01_0003.png 진행중*******
******./train_imgs/BC_01_0004.png 진행중*******
******./train_imgs/BC_01_0005.png 진행중*******
******./train_imgs/BC_01_0006.png 진행중*******
******./train_imgs/BC_01_0007.png 진행중*******
******./train_imgs/BC_01_0008.png 진행중*******
******./train_imgs/BC_01_0009.png 진행중*******
******./train_imgs/BC_01_0010.png 진행중*******
******./train_imgs/BC_01_0012.png 진행중*******
******./train_imgs/BC_01_0013.png 진행중*******
******./train_imgs/BC_01_0014.png 진행중*******


In [ ]:
slide_patches_all[0], slide_patches_size[0]

## 저장하기

In [ ]:
np.save('./array/1123_real_train_1_50.npy', slide_patches_all) 
np.save('./array/1123_real_train_size_1_50.npy', slide_patches_size)